In [4]:
pip install tflearn

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow

  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import cv2                 
import numpy as np        
import os                 
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


In [7]:
IMG_SIZE = 96
LR = 1e-3
nb_classes=10   
MODEL_NAME = 'Urbansign.model'

In [8]:
def cnn_model():
    convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')
    convnet = conv_2d(convnet, 8, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)
    convnet = conv_2d(convnet, 16, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)
    convnet = conv_2d(convnet, 32, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)
    convnet = conv_2d(convnet, 64, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)
    convnet = conv_2d(convnet, 128, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)
    convnet = fully_connected(convnet, 1024, activation='relu')
    convnet = dropout(convnet, 0.6) #Dropout genelde 0.6 ile 0.8 arasında kullanılır.
    convnet = fully_connected(convnet, 10, activation='softmax')
    convnet = regression(convnet, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(convnet, tensorboard_dir='log')
    return model

In [9]:
def one_hot_targets_(Y,nb_classes):
    targets = np.array(Y).reshape(-1)
    print(targets)
    one_hot_targets_ = np.eye(nb_classes)[targets]
    return one_hot_targets_
train_data = np.load('train_data.npy',encoding="latin1",allow_pickle=True)

In [10]:
train = train_data[:]
test = train_data[:500]

In [11]:
print('traindatlen:'+str(len(train)))
print('testdatalen:'+str(len(test)))
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]
Y1=one_hot_targets_(Y,nb_classes)

traindatlen:8732
testdatalen:500
[8 7 4 ... 2 2 5]


In [12]:
print('val y'+str(Y1))
print('len X:'+str(len(X)))
print('len Y:'+str(len(Y)))

val y[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
len X:8732
len Y:8732


In [13]:
test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]
test_y1=one_hot_targets_(test_y,nb_classes)
test_y=test_y1
Y=Y1
print('test_x:'+str(len(test_x)))
print('test_y:'+str(len(test_y)))
print('val y'+str(test_y1))
model=cnn_model()
model.fit({'input': X}, {'targets': Y}, n_epoch=15, validation_set=({'input': test_x}, {'targets': test_y}), 
snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
model.save(MODEL_NAME)
score = model.evaluate(test_x, test_y)
print('Test accuarcy: %0.4f%%' % (score[0] * 100))

Training Step: 2054  | total loss: 0.47177 | time: 11.696s
| Adam | epoch: 015 | loss: 0.47177 - acc: 0.8908 -- iter: 8704/8732
Training Step: 2055  | total loss: 0.44270 | time: 12.780s
| Adam | epoch: 015 | loss: 0.44270 - acc: 0.8954 | val_loss: 0.32200 - val_acc: 0.8900 -- iter: 8732/8732
--
INFO:tensorflow:C:\Users\Eren_\Desktop\Proje\Urbansign.model is not in all_model_checkpoint_paths. Manually adding it.
Test accuarcy: 89.0000%
